# Extent of Occurrence and Area of Occupancy bulk calculator for NatureServe element ranking

Created by Clark Hollenberg at the Colorado Natural Heritage Program, October 2024

This script imports a .csv file with rows containing multi-species point occurrence data and WGS84 lat,lon coordinates. The output is a xlsx file containing extent of occurrence (EOO), area of occupancy (AOO), number of hypothetical EOs, and calculated rarity rank. This enables multi-species calculations which are not currently possible on GeoCAT. The projection coordinate system matches the .prj file from the EOO toolbox provided by IUCN. Calculated values match those of the toolbox, but may differ from GeoCAT by 1% at larger scales.



In [ ]:
# REQUIRED USER INPUT!!!!
##################################################################################

# Define file paths
input_csv = r"C:\Users\chollenb\sample_occurrence_points_to_rank.csv"  
output_xlsx = r"C:\Users\chollenb\EOOAOO_calcs.xlsx"
output_xlsx_ranks = r"C:\Users\chollenb\CalculatedRanks.xlsx"
# Path to the chosen .prj file for Cylindrical Equal Area
projection_prj = r'C:\Users\chollenb\OneDrive - Colostate\Documents\Data\BulkRarityCalc\Cylindrical Equal Area (world).prj'
# Defined ranking rules based on NatureServe rank calculator
rules_df = r"RankingMetricRules.xlsx"
# define separation distance for EO clustering
# note that this is the buffer distance for each observation, so the separation distance is effectively doubled.
eo_separation = "0.5 Kilometers"


# environment setup
###################################################################################

# import necessary modules.
import arcpy
import pandas as pd
import os


print("Loading input points...")
# Read the spreadsheet using pandas
df = pd.read_csv(input_csv)

# Set WGS 84 spatial reference and setup environment
arcpy.env.overwriteOutput = True
sr_wgs84 = arcpy.SpatialReference(4326)  # WGS84
sr_cylindrical_equal_area = arcpy.SpatialReference(projection_prj) # equal area projection

# Create an in-memory feature class from spreadsheet data
# Make sure that your latitude and longitude columns are correct in the function below. Change if necessary
print("Importing input points...")
input_points_fc = "in_memory/occurrence_points"
arcpy.management.XYTableToPoint(
    input_csv, input_points_fc, x_field="decimalLongitude", y_field="decimalLatitude", coordinate_system=sr_wgs84
)

# Project the points to an equal-area projection for area calculations
print("Projecting input points...")
projected_fc = "in_memory/projected_points"
# Use `sr_cylindrical_equal_area` in the projection step
arcpy.management.Project(input_points_fc, projected_fc, sr_cylindrical_equal_area)

# List to store results
results = []

# Identify unique species in input dataset. The species name should be in a column called "SNAME"
species_list = sorted(df["SNAME"].dropna().unique())

# Species processing loop
######################################################################################################

for species in species_list:
    print(f"processing {species}")
    # Select only the points for the current species
    # if it is a species name without variety or subspecies, include all occurrences with varieties derived from the species
    arcpy.management.MakeFeatureLayer(projected_fc, "species_layer", f"SNAME LIKE '{species}%'")
    num_occur = int(arcpy.management.GetCount("species_layer").getOutput(0))

    ####################################################
    # Calculate EOO
    # Generate Minimum Convex Polygon (EOO)
    eoo_fc = os.path.join("in_memory", f"eoo")
    arcpy.management.MinimumBoundingGeometry(
        "species_layer", eoo_fc, "CONVEX_HULL", group_option="ALL"
    )
    
    # Calculate Area for extent of occurrence
    with arcpy.da.SearchCursor(eoo_fc, ["SHAPE@AREA"]) as cursor:
        for row in cursor:
            eoo_area_km2 = row[0] / 1e6  # Convert square meters to square kilometers

    ##############################################
    # Calculate Area of Occupancy

    # Create the 2 km x 2 km grid
    grid_fc = os.path.join("in_memory", f"grid")
    extent = arcpy.Describe("species_layer").extent
    # Snapping to the nearest 2 km grid point
    snapped_x = int(extent.XMin // 2000) * 2000
    snapped_y = int(extent.YMin // 2000) * 2000

    arcpy.management.CreateFishnet(
        grid_fc,
        origin_coord = f"{snapped_x} {snapped_y}",
        y_axis_coord = f"{snapped_x} {snapped_y + 1000}",
        cell_width=2000,
        cell_height=2000,
        number_rows="",
        number_columns="",
        geometry_type="POLYGON",
        template="species_layer"
    )

    # Project the grid to the Cylindrical Equal Area projection
    projected_grid_fc = os.path.join("in_memory", f"projected_grid")
    arcpy.management.Project(grid_fc, projected_grid_fc, sr_cylindrical_equal_area)


    # Spatial Join with the species point layer to determine number of occupied grid cells
    joined_fc = os.path.join("in_memory", f"joined")
    arcpy.analysis.SpatialJoin(projected_grid_fc, "species_layer", joined_fc, "JOIN_ONE_TO_ONE", "KEEP_COMMON", match_option="INTERSECT")
    
    # Count occupied cells
    occupied_cells = len([row for row in arcpy.da.SearchCursor(joined_fc, ["Join_Count"])])

    ##########################################################################################
    # Estimate the number of unique Element Occurences (with 1km separation distance)

    # Create a 1-km buffer around each point
    buffer_fc = os.path.join("in_memory", f"buffer")
    arcpy.analysis.Buffer("species_layer", buffer_fc, eo_separation, dissolve_option="NONE") # adjust separation distance here if desired
    output_fc = os.path.join("in_memory", f"merged")  # The output shapefile or feature class

    # Attribute field to retain after merging
    retain_field = "SNAME" 

    # Perform dissolve to merge intersecting polygons
    arcpy.management.Dissolve(
        in_features=buffer_fc,
        out_feature_class=output_fc,
        dissolve_field=retain_field,  # Field used to retain one polygon's attribute in each merged group
        multi_part="SINGLE_PART"     
    )

    # Count the number of dissolved polygons (clusters)
    num_clusters = int(arcpy.management.GetCount(output_fc).getOutput(0))

    # add results to table and delete feature classes created during calculation.
    results.append({"Species": species, "EOO_Area_km2": eoo_area_km2, "AOO_num_cells": occupied_cells, "num_occur": num_occur, "num_EOs": num_clusters})
    arcpy.management.Delete(buffer_fc)
    arcpy.management.Delete(output_fc)
    arcpy.management.Delete(grid_fc)
    arcpy.management.Delete(projected_grid_fc)
    arcpy.management.Delete(eoo_fc)

# Convert results to DataFrame and export to Excel
result_df = pd.DataFrame(results)
result_df.to_excel(output_xlsx, index=False)

print(f"EOO calculations complete. Results saved to {output_xlsx}")

######################################################################################################################
# Rank estimate based on Calculated EOO/AOO/Num_EOs
rules_df = pd.read_excel(rules_df)
# Function to assign points based on bins in rules spreadsheet
def assign_points(value, ranges_df, metricName):
    for _, row in ranges_df.iterrows():
        if value <= row[f"{metricName}Val"]:
            return row[f"{metricName}Score"]
    return 0  # Default if value is higher than all upper limits

# Apply function to assign points
# AOO is double weighted compared to EOO and num_EOs. Average by 4.
result_df["Points"] = (result_df["EOO_Area_km2"].apply(lambda x: assign_points(x, rules_df, "EOO")) + 2*result_df["AOO_num_cells"].apply(lambda x: assign_points(x, rules_df, "AOO")) + result_df["num_EOs"].apply(lambda x: assign_points(x, rules_df, "Num")))/4
result_df["SRank"] = result_df["Points"].apply(lambda x: assign_points(x, rules_df, "Rank"))

result_df.to_excel(output_xlsx_ranks, index=False)
print(f"Rank calculations complete. Results saved to {output_xlsx}")



Loading input points...
